In [1]:
import pandas as pd
df = pd.read_csv('hospital.csv')

In [2]:
df[:3]

,DAY,Room Setup Start Time,Patient in Room Time,Anesthesia Start Time,Induction End,Procedure Start,Procedure End,Patient out of Room,Surgeon ID,Anesthesiologist ID,Scrub ID,Circulator ID,First Assistant ID,Second Assistant ID
0,1,7:30,9:01,9:01,10:15,9:49,11:19,11:26,190,439,933.0,768,668.0,953.0
1,1,7:30,8:15,8:17,8:36,8:51,9:40,9:45,684,908,993.0,125,639.0,NaN
2,1,7:30,8:05,8:05,8:22,8:37,10:03,10:05,190,362,855.0,520,353.0,953.0


In [3]:
df = df.fillna(999)
df['Scrub ID'] = df['Scrub ID'].astype('int')
df['First Assistant ID'] = df['First Assistant ID'].astype('int')
df['Second Assistant ID'] = df['Second Assistant ID'].astype('int')
df = df.astype('str')

In [4]:
df_gene = df[['DAY','Surgeon ID','Anesthesiologist ID','Scrub ID','Circulator ID','First Assistant ID','Second Assistant ID']]

In [5]:
df_gene[:3]

,DAY,Surgeon ID,Anesthesiologist ID,Scrub ID,Circulator ID,First Assistant ID,Second Assistant ID
0,1,190,439,933,768,668,953
1,1,684,908,993,125,639,999
2,1,190,362,855,520,353,953


In [6]:
gp = df_gene.groupby(['Surgeon ID','Anesthesiologist ID'])['DAY'].count().reset_index().\
rename(index=str, columns={'DAY': 'SA'})
df_gene = df_gene.merge(gp,on=['Surgeon ID','Anesthesiologist ID'],how='left')

In [7]:
df_gene[:3]

,DAY,Surgeon ID,Anesthesiologist ID,Scrub ID,Circulator ID,First Assistant ID,Second Assistant ID,SA
0,1,190,439,933,768,668,953,31
1,1,684,908,993,125,639,999,5
2,1,190,362,855,520,353,953,6


In [8]:
from itertools import combinations
cols = ['Surgeon ID',
       'Anesthesiologist ID', 'Scrub ID', 'Circulator ID',
       'First Assistant ID', 'Second Assistant ID']
for i in list(combinations(cols,2)):
    gp = df_gene.groupby([i[0],i[1]])['DAY'].count().reset_index().\
        rename(index=str, columns={'DAY': str(i[0])[:3]+'_'+str(i[1])[:3]})
    df_gene = df_gene.merge(gp,on=[i[0],i[1]],how='left')

In [9]:
df_gene[:3]

,DAY,Surgeon ID,Anesthesiologist ID,Scrub ID,Circulator ID,First Assistant ID,Second Assistant ID,SA,Sur_Ane,Sur_Scr,...,Ane_Scr,Ane_Cir,Ane_Fir,Ane_Sec,Scr_Cir,Scr_Fir,Scr_Sec,Cir_Fir,Cir_Sec,Fir_Sec
0,1,190,439,933,768,668,953,31,31,6,...,5,18,7,23,6,4,11,4,17,6
1,1,684,908,993,125,639,999,5,5,8,...,11,2,3,24,6,4,9,14,19,3
2,1,190,362,855,520,353,953,6,6,186,...,10,8,3,12,57,14,138,6,82,14


In [10]:
df_gene.columns

Index(['DAY', 'Surgeon ID', 'Anesthesiologist ID', 'Scrub ID', 'Circulator ID',
       'First Assistant ID', 'Second Assistant ID', 'SA', 'Sur_Ane', 'Sur_Scr',
       'Sur_Cir', 'Sur_Fir', 'Sur_Sec', 'Ane_Scr', 'Ane_Cir', 'Ane_Fir',
       'Ane_Sec', 'Scr_Cir', 'Scr_Fir', 'Scr_Sec', 'Cir_Fir', 'Cir_Sec',
       'Fir_Sec'],
      dtype='object')

In [11]:
df = df_gene[ [ 'Sur_Ane', 'Sur_Scr',
       'Sur_Cir', 'Sur_Fir', 'Sur_Sec', 'Ane_Scr', 'Ane_Cir', 'Ane_Fir',
       'Ane_Sec', 'Scr_Cir', 'Scr_Fir', 'Scr_Sec', 'Cir_Fir', 'Cir_Sec',
       'Fir_Sec']]

In [14]:
df_gene['sum']= df.apply(sum,axis=1)
#df_original['DAY'] = df_original['DAY'].astype('int')

In [15]:
df_original = df_gene.groupby('DAY')['sum'].mean().reset_index()

In [16]:
df_original['DAY'] = df_original['DAY'].astype('int')
df_original = df_original.sort_values('DAY').reset_index()

In [17]:
df_original.drop(['index'],axis=1,inplace=True)

In [22]:
best_team = pd.read_csv('BestTeam.csv').rename(index=str, columns={'Day': 'DAY'})
best_team[:3]
best_team = best_team.drop('Unnamed: 0',axis = 1)

In [23]:
df_original = df_original.merge(best_team,on='DAY',how='left')  

In [24]:
df_original = df_original.rename(index=str, columns = {'sum':'Mean'})

In [30]:
import numpy as np
df_original['gap'] = df_original['Cooperate Times'] - df_original['Mean']

In [31]:
df_original[:3]

,DAY,Mean,Teams,Cooperate Times,gap
0,1,754.200000,"[366, 833, 749, 608, 639, 10]",2025,1270.800000
1,2,605.931034,"[743, 465, 40, 701, 510, 889]",2425,1819.068966
2,3,748.629630,"[743, 465, 40, 470, 729, 889]",1326,577.370370


In [170]:
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
df_original.to_csv('Teams.csv')

In [172]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 534 entries, 0 to 533
Data columns (total 5 columns):
DAY               534 non-null int64
Mean              534 non-null float64
Best Team         534 non-null object
Cooperate Time    534 non-null int64
gap               534 non-null float64
dtypes: float64(2), int64(2), object(1)
memory usage: 25.0+ KB
